# Imports

In [37]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal


# Set Paths

In [38]:
# Define the project path and a list of 5 ABF file names
project_path = 'C:\\Users\\ikmor\\OneDrive\\Documents\\SNR AND OBR pilot study\\'

# Load Data

In [39]:

abf_files = [
    f'{project_path}PTX09R5 ACCOM1.abf',
    f'{project_path}PTX09R5 ACCOM2.abf',
    f'{project_path}PTX09R5 ACCOM3.abf',
    f'{project_path}PTX09R5 ACCOM4.abf',
    f'{project_path}PTX09R5 ACCOM5.abf'
]



In [ ]:
import numpy as np
import pyabf
from scipy import signal

# Set the desired sweep and channel
sweep_numbers = [0, 1, 3, 4, 5, 6, 7, 8, 9]

# Input resistance (MegaOhms) - you can change this value as needed
input_resistance_MOhm = float(input("Enter the input resistance in MegaOhms: "))
input_resistance = input_resistance_MOhm * 1e6  # Convert MegaOhms to Ohms

LJP = 13  # Liquid Junction Potential in mV

# Initialize dictionaries to store total power for each sweep and file
total_power_per_sweep = {sweep: [] for sweep in sweep_numbers}
average_total_power_per_file = []

for abf_file in abf_files:
    abf = pyabf.ABF(abf_file)
    file_total_power = []
    
    for sweep in sweep_numbers:
        abf.setSweep(sweep, channel=0)
        # Define the time segment for analysis (in seconds)
        start_time = 0.1  # Start time in seconds
        end_time = 3.0    # End time in seconds

        # Extract voltage (sweepY) and time (sweepX)
        voltage = abf.sweepY - LJP  # Voltage in mV
        times = abf.sweepX    # Time in seconds

        # Find indices corresponding to the desired time segment
        start_index = np.argmin(np.abs(times - start_time))
        end_index = np.argmin(np.abs(times - end_time))

        # Extract the specific portion of the data for analysis
        segment_times = times[start_index:end_index]
        segment_voltage = voltage[start_index:end_index]

        # Convert voltage from mV to V for power calculation
        segment_voltage_V = segment_voltage * 1e-3

        # Calculate instantaneous power using input resistance: P = V^2 / R
        instantaneous_power = (segment_voltage_V ** 2) / input_resistance
        
        # Calculate the Power Spectral Density (PSD) of the instantaneous power using Welch's method with zero padding
        fs = abf.dataRate  # Sampling frequency from ABF file
        segment_length_ms = 50  # Optimal segment length in milliseconds
        nperseg = int(fs * segment_length_ms / 1000)
        noverlap = nperseg // 2  # 50% overlap

        # Ensure nperseg is an even number for efficient FFT
        nperseg = nperseg + (nperseg % 2)

        # Zero padding
        nfft = 320000  # Increase FFT size such that frequency resolution is 0.05Hz

        # Apply Welch's method with zero padding
        f, Pxx = signal.welch(instantaneous_power, fs=fs, window='hann', nperseg=nperseg, noverlap=noverlap, nfft=nfft, scaling='density')

        # Calculate total power excluding frequencies below 0.1 Hz
        freq_mask = (f >= 0.1) & (f <= 10000)
        average_power_above_0_1Hz = np.trapz(Pxx[freq_mask], f[freq_mask])  # Integrate PSD above 0.1 Hz

        # Calculate the duration of the sweep
        sweep_duration = end_time - start_time

        # Calculate total power by multiplying average power by the duration
        total_power_above_0_1Hz = average_power_above_0_1Hz * sweep_duration * 1e24  # Convert to yoctoWatt

        # Add the total power for this sweep to the list
        total_power_per_sweep[sweep].append(total_power_above_0_1Hz)
        file_total_power.append(total_power_above_0_1Hz)
    
    # Calculate average total power for this file
    average_total_power_per_file.append(np.mean(file_total_power))

# Calculate average total power for each sweep across all files
average_total_power_per_sweep = {sweep: np.mean(powers) for sweep, powers in total_power_per_sweep.items()}

# Calculate the average of the average total power across all files
overall_average_total_power = np.mean(average_total_power_per_file)

# Print results
for sweep in sweep_numbers:
    print(f"Average total power for sweep {sweep}: {average_total_power_per_sweep[sweep]:.6e} yoctoWatt")

print(f"\nAverage of the average total power across all files: {overall_average_total_power:.6e} yoctoWatt")

Average total power for sweep 0: 9.623003e-01 yoctoWatt
Average total power for sweep 1: 9.675156e-01 yoctoWatt
Average total power for sweep 3: 9.640149e-01 yoctoWatt
Average total power for sweep 4: 9.651447e-01 yoctoWatt
Average total power for sweep 5: 9.686868e-01 yoctoWatt
Average total power for sweep 6: 9.538383e-01 yoctoWatt
Average total power for sweep 7: 9.704044e-01 yoctoWatt
Average total power for sweep 8: 9.745984e-01 yoctoWatt
Average total power for sweep 9: 9.776388e-01 yoctoWatt

Average of the average total power across all files: 9.671269e-01 yoctoWatt
